# Lectura: Mejorando el Rendimiento de Consultas Lentas en MySQL

En esta lectura, aprenderás cómo mejorar el rendimiento de consultas lentas en MySQL.

## Objetivos

Después de completar esta lectura, podrás:

1. Describir las razones comunes de las consultas lentas en MySQL
2. Identificar la razón del rendimiento de tu consulta con la instrucción `EXPLAIN`
3. Mejorar el rendimiento de tu consulta con índices y otras mejores prácticas

## Software Utilizado

En esta lectura, verás el uso de [MySQL](https:\www.mysql.com\). MySQL es un Sistema de Gestión de Bases de Datos Relacional (RDBMS) diseñado para almacenar, manipular y recuperar datos de manera eficiente.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Reading%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/mysql.png)

## Causas Comunes de Consultas Lentas

A veces, cuando ejecutas una consulta, podrías notar que la salida aparece mucho más lenta de lo que esperabas, tardando unos segundos, minutos o incluso horas en cargar. ¿Por qué podría estar sucediendo esto?

Hay muchas razones para una consulta lenta, pero algunas comunes incluyen:

1. El tamaño de la base de datos, que está compuesto por el número de tablas y el tamaño de cada tabla. Cuanto más grande sea la tabla, más tiempo tardará una consulta, especialmente si estás realizando escaneos de toda la tabla cada vez.
    
2. Consultas no optimizadas pueden llevar a un rendimiento más lento. Por ejemplo, si no has indexado correctamente tu base de datos, los resultados de tus consultas cargarán mucho más lento.
    

Cada vez que ejecutas una consulta, verás una salida similar a la siguiente:

![Sample SELECT Output](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Reading%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/Sample_SEC_OUTPUT.png)

Como se puede ver, la salida incluye el número de filas devueltas y cuánto tiempo tardó en ejecutarse, dado en el formato de `0.00` segundos.

Una herramienta incorporada que se puede usar para determinar por qué tu consulta podría estar tardando más tiempo en ejecutarse es la declaración `EXPLAIN`.

## EXPLICAR el Rendimiento de Tu Consulta

La declaración `EXPLAIN` proporciona información sobre cómo MySQL ejecuta tu declaración, es decir, cómo MySQL planea ejecutar tu consulta. Con `EXPLAIN`, puedes verificar si tu consulta está obteniendo más información de la que necesita, lo que resulta en un rendimiento más lento debido al manejo de grandes cantidades de datos.

Esta declaración funciona con `SELECT`, `DELETE`, `INSERT`, `REPLACE` y `UPDATE`. Cuando se ejecuta, genera una tabla que se ve como la siguiente:

![Muestra de Salida de EXPLAIN](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Reading%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/Sample_EXPLAIN.png)

Como se muestra en la tabla de salida, con `SELECT`, la declaración `EXPLAIN` te indica qué tipo de selección realizaste, la tabla sobre la que se está realizando la selección, el número de filas examinadas y cualquier información adicional.

En este caso, la declaración `EXPLAIN` nos mostró que la consulta realizó una selección simple (en lugar de, por ejemplo, una subconsulta o una selección de unión) y que se examinaron 298,980 filas (de un total de aproximadamente 300,024 filas).

El número de filas examinadas puede ser útil para determinar por qué una consulta es lenta. Por ejemplo, si notas que tu salida es solo de 13 filas, pero la consulta está examinando alrededor de 300,000 filas—¡casi toda la tabla!—entonces eso podría ser una razón para el rendimiento lento de tu consulta.

En el ejemplo anterior, cargar alrededor de 300,000 filas tomó menos de un segundo para procesar, por lo que eso puede no ser una gran preocupación con esta base de datos. Sin embargo, ese puede no ser el caso con bases de datos más grandes que pueden tener hasta un millón de filas.

Un método para hacer que estas consultas sean más rápidas es agregando índices a tu tabla.

## Indexando una Columna

Piensa en los índices como marcadores. Los índices apuntan a filas específicas, ayudando a la consulta a determinar qué filas coinciden con sus condiciones y recuperando esos resultados rápidamente. Con este proceso, la consulta evita buscar a través de toda la tabla y mejora el rendimiento de tu consulta, particularmente cuando usas cláusulas SELECT y WHERE.

Hay muchos tipos de índices que puedes agregar a tus bases de datos, siendo los más populares los índices regulares, índices primarios, índices únicos, índices de texto completo e índices de prefijo.

| Tipo de Índice | Descripción |
| --- | --- |
| Índice Regular | Un índice donde los valores no tienen que ser únicos y pueden ser NULL. |
| Índice Primario | Los índices primarios se crean automáticamente para claves primarias. Todos los valores de columna son únicos y no se permiten valores NULL. |
| Índice Único | Un índice donde todos los valores de columna son únicos. A diferencia del índice primario, los índices únicos pueden contener un valor NULL. |
| Índice de Texto Completo | Un índice utilizado para buscar a través de grandes cantidades de texto y solo se puede crear para columnas de tipo **char**, **varchar** y/o **text**. |
| Índice de Prefijo | Un índice que utiliza solo los primeros N caracteres de un valor de texto, lo que puede mejorar el rendimiento ya que solo esos caracteres tendrían que ser buscados. |

Ahora, podrías preguntarte: si los índices son tan buenos, ¿por qué no los agregamos a cada columna?

En general, es una buena práctica evitar agregar índices a todas tus columnas, solo agregándolos a aquellas que pueden ser útiles, como una columna que se accede con frecuencia. Si bien la indexación puede mejorar el rendimiento de algunas consultas, también puede ralentizar tus inserciones, actualizaciones y eliminaciones porque cada índice necesitará ser actualizado cada vez. Por lo tanto, es importante encontrar el equilibrio entre el número de índices y la velocidad de tus consultas.

Además, los índices son menos útiles para consultar tablas pequeñas o tablas grandes donde casi todas las filas necesitan ser examinadas. En el caso donde la mayoría de las filas necesitan ser examinadas, sería más rápido leer todas esas filas en lugar de usar un índice. Así, agregar un índice depende de tus necesidades.

## Sé SELECTivo con las Columnas

Cuando sea posible, evita seleccionar todas las columnas de tu tabla. Con conjuntos de datos más grandes, seleccionar todas las columnas y mostrarlas puede tardar mucho más que seleccionar una o dos columnas que realmente necesitas.

Por ejemplo, con un conjunto de datos de aproximadamente 300,000 entradas de empleados, la siguiente consulta tarda alrededor de 0.31 segundos en cargar:

```
SELECT * FROM employee

```

![Muestra de la salida SELECT ALL](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Reading%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/Sample_SELECT_ALL_Output.png)

Pero si solo quisiéramos ver los números de empleado y sus fechas de contratación (2 de las 6 columnas), podríamos hacerlo fácilmente con esta consulta que tarda 0.12 segundos en cargar:

```
SELECT employee_number, hire_date FROM employee;

```

![Salida de ejemplo de SELECT con número de empleado y fecha de contratación](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Reading%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/Sample_SELECTive_Output.png)

Nota cómo el tiempo de ejecución de la consulta es mucho más rápido en comparación con cuando seleccionamos todos. Este método puede ser útil al tratar con grandes conjuntos de datos de los que solo necesitas seleccionar columnas específicas.

## Evitar comodines iniciales

Los comodines iniciales, que son comodines (`"%abc"`) que encuentran valores que terminan con caracteres específicos, resultan en escaneos completos de la tabla, incluso con índices en su lugar.

Si tu consulta utiliza un comodín inicial y tiene un rendimiento deficiente, considera usar un índice de texto completo en su lugar. Esto mejorará la velocidad de tu consulta mientras evitas la necesidad de buscar en cada fila.

## Usar la cláusula UNION ALL

Al usar el `OR` operador con declaraciones `LIKE`, una cláusula `UNION ALL` puede mejorar la velocidad de tu consulta, especialmente si las columnas en ambos lados del operador están indexadas.

Esta mejora se debe a que el operador `OR` a veces escanea toda la tabla y pasa por alto los índices, mientras que el operador `UNION ALL` los aplicará a las declaraciones `SELECT` separadas.

# Laboratorio Práctico: Mejorando el Rendimiento de Consultas Lentas en MySQL

En este laboratorio, aprenderás cómo mejorar el rendimiento de tus consultas lentas en MySQL, lo cual puede ser particularmente útil con bases de datos grandes.

# Objetivos

Después de completar este laboratorio, podrás:

1. Usar la declaración `EXPLAIN` para verificar el rendimiento de tu consulta
2. Agregar índices para mejorar el rendimiento de tu consulta
3. Aplicar otras mejores prácticas, como utilizar la cláusula `UNION ALL` para mejorar el rendimiento de la consulta

# Base de Datos Utilizada en este Laboratorio

La base de datos Employees utilizada en este laboratorio proviene de la siguiente fuente: [https://dev.mysql.com/doc/employee/en/](https://dev.mysql.com/doc/employee/en/) bajo la [Licencia CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/).

El siguiente diagrama de entidad-relación (ERD) muestra el esquema de la base de datos Employees:

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/employees-schema.png)

La primera fila de cada tabla es el nombre de la tabla, las filas con claves al lado indican las claves primarias, y las filas restantes son atributos adicionales.

# Ejercicio 1: Cargar la Base de Datos

Comencemos por recuperar la base de datos y cargarla para que pueda ser utilizada.

1. En la barra de menú, selecciona `Terminal > New Terminal`. Esto abrirá el Terminal.
    
    Para descargar el archivo zip que contiene la base de datos, copia y pega lo siguiente en el Terminal:
    
    ```bash
    wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/datasets/employeesdb.zip
    ```
    
    ![Descargar Base de Datos](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/a-download_database.png)
    
2. A continuación, necesitaremos descomprimir su contenido. Podemos hacerlo con el siguiente comando:
    
    ```bash
    unzip employeesdb.zip
    ```
    
    ![Descomprimiendo la Base de Datos Descargada](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/a-unzip_database.png)
    
3. Ahora, cambiemos de directorio para que podamos acceder a los archivos en la nueva carpeta **employeesdb**.
    
    ```bash
    cd employeesdb
    ```
    
    Verifica la línea junto a **theia@theiadocker**. Si dice **/home/project/employeesdb**, ¡entonces has cambiado de directorio con éxito!
    
    ![Cambiando de Directorios](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/a-change_directories.png)
    
4. Inicia la sesión del servicio MySQL utilizando la directiva `Start MySQL in IDE button`.
    

  

5. En la página de inicio, haz clic en el botón **Crear**.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/0fIjtcRkAkn5aEwr-GfvyQ/k2-1.png)

6. Con tu contraseña a la mano, ahora podemos importar los datos. Puedes hacerlo ingresando lo siguiente en el Terminal:
    
    ```bash
    mysql --host=mysql --port=3306 --user=root --password -t < employees.sql
    ```
    

Cuando se te pida, ingresa la contraseña que se mostró en la sección **Información de Conexión** cuando MySQL se inició.

> Ten en cuenta que no podrás ver tu contraseña mientras la escribes. ¡No te preocupes, esto es normal!

7. Tus datos se cargarán ahora. Esto puede tardar un minuto o más.
    
    Cuando hayas terminado de cargar los datos, verás lo siguiente:
    

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/Dvc2ViiLfT5haE01N4fYug/A6.PNG)

Esto significa que tus datos han sido importados.

8. Para entrar en la interfaz de línea de comandos de MySQL, regresa a tu pestaña de MySQL y selecciona **MySQL CLI**.
    
    ![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/PY0u8YRYSB9F69VA4r8uBg/k2-4.png)
    
9. Recuerda que el nombre de la base de datos que estamos utilizando es **Employees**. Para acceder a ella, podemos usar este comando:
    
    ```bash
    use employees
    ```
    
    ![Usar Base de Datos Employees](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/a-database_changed.png)
    
10. Veamos qué tablas están disponibles en esta base de datos:
    
    ```bash
    show tables;
    ```
    
    ![Mostrar Tablas en la Base de Datos Employees](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/a-show_tables.png)
    
    En esta base de datos, hay 8 tablas, lo cual podemos confirmar con el ERD de la base de datos.
    

Ahora que tu base de datos está configurada, ¡veamos cómo podemos verificar el rendimiento de una consulta!

# Ejercicio 2: Verifica el Rendimiento de tu Consulta con EXPLAIN

La declaración `EXPLAIN`, que proporciona información sobre cómo MySQL ejecuta tu declaración, te ofrecerá información sobre el número de filas que tu consulta planea revisar. Esta declaración puede ser útil cuando tu consulta se está ejecutando lentamente. Por ejemplo, ¿se está ejecutando lentamente porque está escaneando toda la tabla cada vez?

1. Comencemos seleccionando todos los datos de la tabla **employees**:
    
    ```
    SELECT * FROM employees;
    
    ```
    
    ![Select All Output](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/b-select_all_output.png)
    
    Como puedes ver, se cargaron todas las 300,024 filas, tomando aproximadamente 0.34 segundos.
    
2. Podemos usar `EXPLAIN` para ver cuántas filas fueron escaneadas:
    
    ```
    EXPLAIN SELECT * FROM employees;
    
    ```
    
    ![EXPLAIN Select All Output](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/b-explain_select_all_output.png)
    
    ¡Observa cómo `EXPLAIN` muestra que está examinando 298,980 filas, casi toda la tabla! Con una tabla más grande, esto podría resultar en que la consulta se ejecute lentamente.
    

> Entonces, ¿cómo podemos hacer que esta consulta sea más rápida? ¡Ahí es donde entran los índices!

# Ejercicio 3: Agregar un Índice a Tu Tabla

1. Para comenzar, veamos los índices existentes. Podemos hacer eso ingresando el siguiente comando:
    
    ```
    SHOW INDEX FROM employees;
    
    ```
    
    ![Índices de Clave Primaria](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-show_indexes_primarykey.png)
    
    Recuerda que los índices para claves primarias se crean automáticamente, como podemos ver arriba. Ya se ha creado un índice para la clave primaria, **emp\_no**. Si lo pensamos, tiene sentido porque cada número de empleado es único para el empleado, sin valores NULL.
    
2. Ahora, digamos que queremos ver toda la información sobre los empleados que fueron contratados el 1 de enero de 2000 o después. Podemos hacer eso con la consulta:
    
    ```
    SELECT * FROM employees WHERE hire_date >= '2000-01-01';
    
    ```
    
    ![Seleccionar Todas las Columnas con Fecha de Contratación Antes del 1 de Enero de 2000](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-sample_hiredate.png)
    
    Como podemos ver, las 13 filas devueltas tardaron aproximadamente 0.17 segundos en ejecutarse. Eso puede no parecer mucho tiempo con esta tabla, pero ten en cuenta que con tablas más grandes, este tiempo puede variar mucho.
    
3. Con la declaración `EXPLAIN`, podemos verificar cuántas filas está escaneando esta consulta:
    
    ```
    EXPLAIN SELECT * FROM employees WHERE hire_date >= '2000-01-01';
    
    ```
    
    ![EXPLAIN Seleccionar Todas las Columnas con Fecha de Contratación Antes del 1 de Enero de 2000](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-sample_explain_hiredate.png)
    
    Esta consulta resulta en un escaneo de 299,423 filas, ¡casi toda la tabla!
    
    Al agregar un índice a la columna **hire\_date**, podremos reducir la necesidad de la consulta de buscar a través de cada entrada de la tabla, buscando solo lo que necesita.
    
4. Puedes agregar un índice con lo siguiente:
    
    ```
    CREATE INDEX hire_date_index ON employees(hire_date);
    
    ```
    
    El comando `CREATE INDEX` crea un índice llamado **hire\_date\_index** en la tabla **employees** en la columna **hire\_date**.
    
    ![Crear un Índice en la Fecha de Contratación](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-create_hiredate_index.png)
    
5. Para verificar tu índice, puedes usar el comando `SHOW INDEX`:
    
    ```
    SHOW INDEX FROM employees;
    
    ```
    
    Ahora puedes ver que tenemos tanto el índice **emp\_no** como el índice **hire\_date**.
    
    ![Mostrar Índices de Clave Primaria y Nueva Fecha de Contratación](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-show_index_primarykey_and_hiredateindex.png)
    
    Con el índice agregado,
    
6. Una vez más, seleccionemos a todos los empleados que fueron contratados el 1 de enero de 2000 o después.
    
    ```
    SELECT * FROM employees WHERE hire_date >= '2000-01-01';
    
    ```
    
    ![Seleccionar Todas las Columnas con Fecha de Contratación Antes del 1 de Enero de 2000 con Índice](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-sample_hiredate_with_index.png)
    
    ¡La diferencia es bastante evidente! En lugar de tardar aproximadamente 0.17 segundos en ejecutar la consulta, toma 0.00 segundos—casi ningún tiempo.
    
7. Podemos usar la declaración `EXPLAIN` para ver cuántas filas se escanearon:
    
    ```
    EXPLAIN SELECT * FROM employees WHERE hire_date >= '2000-01-01';
    
    ```
    
    ![EXPLAIN Seleccionar Todas las Columnas con Fecha de Contratación Antes del 1 de Enero de 2000 con Índice](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-sample_explain_hiredate_with_index.png)
    
    Bajo **rows**, podemos ver que solo se escanearon las 13 columnas necesarias, lo que llevó a una mejora en el rendimiento.
    
    Bajo **Extra**, también puedes ver que se ha declarado explícitamente que se utilizó el índice, siendo ese índice **hire\_date\_index** basado en la columna **possible\_keys**.
    

Ahora, si deseas eliminar el índice, ingresa lo siguiente en la Terminal:

```
DROP INDEX hire_date_index ON employees;

```

Esto eliminará el **hire\_date\_index** de la tabla **employees**. Puedes verificarlo con el comando `SHOW INDEX` para confirmar:

![Eliminar el índice de fecha de contratación y mostrar los índices actuales en la tabla de empleados](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/c-drop_and_show_index.png)

# Ejercicio 4: Utilizar una cláusula UNION ALL

A veces, puede que desees ejecutar una consulta utilizando el operador `OR` con declaraciones `LIKE`. En este caso, usar una cláusula `UNION ALL` puede mejorar la velocidad de tu consulta, especialmente si las columnas a ambos lados del operador `OR` están indexadas.

1. Para comenzar, ejecutemos esta consulta:
    
    ```bash
    SELECT * FROM employees WHERE first_name LIKE 'C%' OR last_name LIKE 'C%';
    ```
    
    ![Consulta OR de Muestra](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-sample_or_query.png)
    
    Esta consulta busca nombres o apellidos que comiencen con “C”. Devolvió 28,970 filas, tomando aproximadamente 0.20 segundos.
    
2. ¡Verifica usando el comando `EXPLAIN` para ver cuántas filas se están escaneando!
    
   <details><summary>Pista (Haz clic aquí) </summary>
    
    Revisa cómo utilizamos la declaración `EXPLAIN` en el Ejercicio A y aplícalo a la consulta anterior.
    </details>
    <br>
    <details><summary>Solución (Haz clic aquí)</summary>
    
    Tu declaración debería verse así:
    
    ```
    EXPLAIN SELECT * FROM employees WHERE first_name LIKE 'C%' OR last_name LIKE 'C%';
    ```
    
    ![Consulta OR de Muestra con EXPLAIN](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-explain_sample_or_query.png)

    </details>
    
    Una vez más, podemos ver que casi todas las filas están siendo escaneadas, así que agreguemos índices a las columnas **first\_name** y **last\_name**.
    
3. Intenta agregar un índice a las columnas **first\_name** y **last\_name**.
    
    <details><summary>Pista (Haz clic aquí)</summary>
    
    Considera cómo creamos un índice en el Ejercicio B. ¿Cómo puedes aplicar esto a una columna diferente?
    </details>
    <br>
    <details><summary>Solución (Haz clic aquí)</summary>
    
    Puedes agregar los índices con lo siguiente:
     
    ```bash
    CREATE INDEX first_name_index ON employees(first_name);CREATE INDEX last_name_index ON employees(last_name);
    ```
    
    Ten en cuenta que el nombre de tus índices (**first\_name\_index** y **last\_name\_index**) puede ser diferente.
    
    También puedes verificar si tus índices han sido añadidos con el comando `SHOW INDEX`:
    
    ![Agregar un Índice al Nombre y Apellido y Mostrar Índices Actuales](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-add_indexes_show_index.png)
    
    </details>


4. ¡Genial! Con tus índices ahora en su lugar, podemos volver a ejecutar la consulta:
    
    ```bash
    SELECT * FROM employees WHERE first_name LIKE 'C%' OR last_name LIKE 'C%';
    ```
    
    ![Reejecutando Consulta OR de Muestra](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-sample_or_query_rerun.png)
    
    También veamos cuántas filas están siendo escaneadas:
    
    ```bash
    EXPLAIN SELECT * FROM employees WHERE first_name LIKE 'C%' OR last_name LIKE 'C%';
    ```
    
    ![EXPLAIN Reejecutando Consulta OR de Muestra](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-explain_sample_or_query_rerun.png)
    
    Con índices, la consulta aún escanea todas las filas.
    
5. Utilicemos la cláusula `UNION ALL` para mejorar el rendimiento de esta consulta.
    
    Podemos hacerlo con lo siguiente:
    
    ```bash
    SELECT * FROM employees WHERE first_name LIKE 'C%' UNION ALL SELECT * FROM employees WHERE last_name LIKE 'C%';
    ```
    
    ![Consulta UNION ALL de Muestra](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-sample_union_all_query.png)
    
    Como podemos ver, esta consulta solo toma 0.11 segundos en ejecutarse, siendo más rápida que cuando usamos el operador `OR`.
    
    Usando la declaración `EXPLAIN`, podemos ver por qué podría ser así:
    
    ![Consulta EXPLAIN UNION ALL de Muestra](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-explain_sample_union_all_query.png)
    
    Como revela la declaración `EXPLAIN`, se realizaron dos operaciones `SELECT`, con el número total de filas escaneadas siendo 54,790. Esto es menos que la consulta original que escaneó toda la tabla y, como resultado, la consulta se ejecuta más rápido.
    

Ten en cuenta que, si decides realizar una búsqueda con comodín inicial con un índice, toda la tabla aún será escaneada. Puedes ver esto tú mismo con la siguiente consulta:

```bash
SELECT * FROM employees WHERE first_name LIKE '%C';
```

Con esta consulta, queremos encontrar todos los empleados cuyos nombres de pila terminan con “C”.

Al verificar con las declaraciones `EXPLAIN` y `SHOW INDEX`, podemos ver que aunque tenemos un índice en **first\_name**, el índice no se utiliza y resulta en una búsqueda de toda la tabla.

En la columna **possible\_keys** de la declaración `EXPLAIN`, podemos ver que este índice no se ha utilizado ya que la entrada es NULL.

![Consulta de comodín inicial de muestra](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-sample_leading_wildcard_query.png)

Por otro lado, los índices sí funcionan con comodines finales, como se ve en la siguiente consulta que encuentra todos los empleados cuyos nombres de pila comienzan con “C”:

```bash
SELECT * FROM employees WHERE first_name LIKE 'C%';
```

![Consulta de comodín final de muestra](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/d-sample_trailing_wildcard_query.png)

Bajo la declaración `EXPLAIN`, en las columnas **possible\_keys** y **Extra**, podemos ver que se utiliza el **first\_name\_index**. Con solo 20,622 filas escaneadas, la consulta tiene un mejor rendimiento.

# Ejercicio 5: SÉ SELECTIVO

En general, es una buena práctica seleccionar solo las columnas que necesitas. Por ejemplo, si quisieras ver los nombres y las fechas de contratación de los diversos empleados, podrías mostrar eso con la siguiente consulta:

```
SELECT * FROM employees;

```

![Muestra Seleccionar Todo de la Tabla de Empleados](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/e-sample_select_all_query.png)

Nota cómo la consulta carga 300,024 filas en aproximadamente 0.26 segundos. Con la instrucción `EXPLAIN`, podemos ver que se está escaneando toda la tabla, lo cual tiene sentido porque estamos viendo todas las entradas.

Sin embargo, si solo quisiéramos ver los nombres y las fechas de contratación, entonces deberíamos seleccionar esas columnas:

```
SELECT first_name, last_name, hire_date FROM employees;

```

![Muestra de Nombre, Apellido y Fecha de Contratación de la Tabla de Empleados](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/e-select_specific_columns_query.png)

Como puedes ver, esta consulta se ejecutó un poco más rápido a pesar de escanear toda la tabla también.

¡Inténtalo!

## Ejercicio Práctico 1

Echemos un vistazo a la tabla de **salaries**. ¿Qué pasaría si quisiéramos ver cuánto gana cada empleado?

Al ejecutar la consulta, ten en cuenta cuánto tiempo tarda en ejecutarse y cuántas filas se escanean cada vez.

1. Primero, seleccionemos todas las filas y columnas de esta tabla.
    
    <details><summary>Pista (Haz clic aquí) </summary>
        
    Necesitarás dos consultas separadas: una para ver la consulta y la salida, y otra para ver cuántas filas se procesan.
    </details>

    <br>
    <details><summary>Solución (Haz clic aquí) </summary>
    
    Para seleccionar todas las filas y columnas, utilizaremos la siguiente consulta:
    
    ```
    SELECT * FROM salaries;
    
    ```
    
    Aunque el tiempo exacto puede variar, en este caso, tardó aproximadamente 1.71 segundos en cargar 2,844,047 filas.
    
    Podemos verificar cuántas filas se escanearon con la siguiente declaración:
    
    ```
    EXPLAIN SELECT * FROM salaries;
    
    ```
    
    Podemos ver que casi toda la tabla fue escaneada, como se esperaba, totalizando 2,838,426 filas.
    
    ![Salida del Ejercicio Práctico 1 Parte 1](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/e-practice_exercise1_output.png)

    </details>
    
2. Ahora, veamos si hay una manera de optimizar esta consulta. Dado que solo queremos ver cuánto gana cada empleado, podemos seleccionar solo algunas columnas en lugar de todas. ¿Cuáles seleccionarías?
    
    <details><summary>Pista (Haz clic aquí) </summary>
    
    Necesitarás dos consultas separadas: una para ver la consulta y la salida, y otra para ver cuántas filas se procesan. Considera las columnas en esta tabla: **emp\_no**, **salary**, **from\_date** y **to\_date**.
    
    </details>
    <br>
    <details><summary>Solución (Haz clic aquí) </summary>
    
    Para seleccionar columnas que nos den información sobre el empleado y su salario correspondiente, elegiremos las columnas **emp\_no** y **salary** con la siguiente consulta:
    
    ```
    SELECT emp_no, salary FROM salaries;
    
    ```
    
    Aunque el tiempo exacto puede variar, en este caso, tardó aproximadamente 1.19 segundos en cargar 2,844,047 filas.
    
    Podemos verificar cuántas filas se escanearon con la siguiente declaración:
    
    ```
    EXPLAIN  SELECT emp_no, salary FROM salaries;
    
    ```
    
    Podemos ver que casi toda la tabla fue escaneada, como se esperaba, totalizando 2,838,426 filas. Sin embargo, se cargó más rápido que en la primera instancia porque fuimos más selectivos en las columnas elegidas.
    
    ![Salida del Ejercicio Práctico 1 Parte 2](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/e-practice_exercise1-2_output.png)
    
    </details>

## Ejercicio Práctico 2

Echemos un vistazo a la tabla de **titles**. ¿Qué pasaría si quisiéramos ver al empleado y su título correspondiente?

- ¡Practica seleccionando solo las columnas necesarias y ejecuta la consulta!

    <details><summary>Pista (Haz clic aquí) </summary>

    Necesitarás dos consultas separadas: una para ver la consulta y la salida, y otra para ver cuántas filas se procesan. Considera las columnas en esta tabla: **emp\_no**, **title**, **from\_date** y **to\_date**.

    </details>
    <br>
    <details><summary>Solución (Haz clic aquí) </summary>

    Para seleccionar columnas que nos den información sobre el empleado y su título correspondiente, elegiremos las columnas **emp\_no** y **title** con la siguiente consulta:

    ```bash
    SELECT emp_no, title FROM titles;

    ```

    Although the exact time may differ, in this instance, it took about 0.22 seconds to load 443,308 rows.

    We can check how many rows were scanned with the following statement:

    ```
    EXPLAIN  SELECT emp_no, title FROM titles;

    ```

    Podemos ver que casi toda la tabla fue escaneada, como se esperaba, sumando un total de 442,545 filas.

    ![Salida del Ejercicio Práctico 2](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/e-practice_exercise2_output.png)

    En comparación, si hubieras ejecutado esto con todas las columnas seleccionadas, es posible que hayas notado que tomó alrededor de 0.47 segundos cargar y escanear la misma cantidad de filas:

    ![Salida del Ejercicio Práctico 2](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Improving%20Performance%20of%20Slow%20Queries%20in%20MySQL/images/e-practice_exercise2_output_selectall.png)

    </details>

# Conclusión

¡Felicidades! Ahora, no solo puedes identificar las causas comunes de las consultas lentas, sino que también puedes resolverlas aplicando el conocimiento que has adquirido en este laboratorio. Equipado con esta habilidad para resolver problemas, podrás mejorar el rendimiento de tus consultas, incluso en bases de datos grandes.